### nn은 모델을 정의하고 미분하는데 autograd 사용.
### nn.Module은 layer와 output을 반환하는 forward(input) 매서드를 포함하고 있음

신경망의 일반적인 학습 과정은 다음과 같습니다:

*학습 가능한 매개변수(또는 가중치(weight))를 갖는 신경망을 정의합니다.

*데이터셋(dataset) 입력을 반복합니다.

*입력을 신경망에서 전파(process)합니다.

*손실(loss; 출력이 정답으로부터 얼마나 떨어져 있는지)을 계산합니다.

*변화도(gradient)를 신경망의 매개변수들에 역으로 전파합니다.

*신경망의 가중치를 갱신합니다. 일반적으로 : 새로운 가중치(weight) = 가중치(weight) - 학습률(learning rate) * 변화도(gradient)

In [1]:
# 신경망 정의

import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 입력 이미지 채널 1개(흑백), 출력 채널 6개, 5x5 크기의 필터(커널)
        self.conv1 = nn.Conv2d(1, 6, 5)
        # 입력 이미지 채널 6개, 출력 채널 16개, 5x5 크기의 필터(커널)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # 아핀(affine) 연산: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 5*5은 이미지 차원, 120는 출력 노드 개수
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)


    def forward(self, input):
        # c는 convolution(합성곱)-이미지 특징 추출, s는 subsampling(pooling) - 특징 맵 크기를 줄임
        c1 = F.relu(self.conv1(input))
        s2 = F.max_pool2d(c1, (2, 2))
        c3 = F.relu(self.conv2(s2))
        s4 = F.max_pool2d(c3, 2)
        s4 = torch.flatten(s4, 1)
        # f는 fully-connect(완전 연걸) - 신경망의 마지막 부분(분류를 담당)
        f5 = F.relu(self.fc1(s4))
        f6 = F.relu(self.fc2(f5))

        output = self.fc3(f6)
        
        return output


net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하면 backward 함수는 autograd가 자동으로 정의됨.

In [2]:
# 학습 가능한 매개변수들은 net.parameters()에 의해 반환
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1의 .weight

10
torch.Size([6, 1, 5, 5])


In [3]:
# 해당 net의 입력 크기는 32x32, 32x32로 크기 변경
input = torch.randn(1, 1, 32, 32) # 배치 사이즈, 채널 수, 높이, 너비
out = net(input)
print(out)

tensor([[ 0.0315,  0.0495, -0.0393,  0.0129,  0.1233, -0.0307, -0.0945, -0.0781,
         -0.0421,  0.0791]], grad_fn=<AddmmBackward0>)


In [4]:
# 모든 매개변수 gradient buffer를 0으로 설정, 무작위 값으로 역전파
net.zero_grad()
out.backward(torch.randn(1,10)) # 스칼라 값이 아니여서 가중치 지정

###요약
*torch.Tensor - autograd 연산을 지원하는 다차원 배열, tensor에 대한 변화도를 가지고 있음
*nn.Module - 신경망 모듈, 매개변수를 캡슐화하는 간편한 방법
*nn.Parameter - Tensor의 한 종류로, Module에 속성으로 할당될 때 자동으로 매개변수로 등록된다.
*autograd.Function - autograd 연산의 순방향과 역방향 정의 를 구현합니다.

# 손실 함수는 
(output, target)을 한 쌍의 입력으로 받아, 출력(output)과 정답(target)이 얼마나 떨어져 있는지 추정하는 값을 계산

In [6]:
#nn 패키지에는 여러 손실함수를 지원한다. 간단한 평균제곱오차를 사용 nn.MSEloss
output = net(input)
target = torch.randn(10) # 랜덤 정답
target = target.view(1,-1) # output과 같은 shape
criterion = nn.MSELoss()

loss = criterion(output, target) # loss 함수 계산
print(loss)

tensor(0.8746, grad_fn=<MseLossBackward0>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

# 역전파는 loss.backward만 하면됨. 기존 변화도를 0으로 만들어야함.

In [8]:
# loss.backward() 호출
net.zero_grad() # 모든 매개변수 변화도 0으로 초기화

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0076, -0.0006,  0.0073, -0.0076, -0.0039, -0.0078])


# 가중치 갱신
가장 많이 사용되는 가장 단순한 규칙은 확률적 경사하강법(SGD)이다.

weigh(새로운 가중치) = weight(가중치) - learning_rate(학습률) * gradient(변화도)

In [9]:
# 확률적 경사하강법 구현
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [10]:
# 여러 optimizer 지원
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정
optimizer.zero_grad()   # 변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # 업데이트 진행